In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from collections import OrderedDict 
import redis

import sys
from paths import Paths
paths = Paths()
sys.path.append(paths.scripts_root)
import util

%load_ext autoreload
%autoreload 2

# Display entire width of browser
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

sys.path.append(paths.inhouse+'/dif/python')
import dif_utils as du

# from Yesh's DIF_IP script
from pixval import Formatter
import dif_utils
from enum import IntEnum
from skimage import measure

green = (0, 255, 0)

paths.print_paths()

Paths defined in paths module:
 paths.data_root: /Users/trafferty/data
 paths.notebook_root: /Users/trafferty/data/notebooks
 paths.scripts_root: /Users/trafferty/data/scripts
 paths.images_root: /Users/trafferty/data/images
 paths.projects_root: /Users/trafferty/data/projects
 paths.results_root: /Users/trafferty/data/results
 paths.tmp_root: /Users/trafferty/data/tmp
 paths.dat_root: /Users/trafferty/data/dat
 paths.tmp_dir: /Users/trafferty/tmp
 paths.ws_dir: /Users/trafferty/workspace
 paths.mii_home: /Users/trafferty/dev
 paths.inhouse: /Users/trafferty/dev/src/inhouse


In [2]:
try:
    r = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)
    if r.ping():
        print("Connected to server")
except:
    print("Error: no connection to server")

Connected to server


In [3]:
# Fetch all completed experiments
#
completed_list = list(r.smembers('experiments_completed_set'))

In [4]:
# Set some variables, keys, etc
#
num_bins = 30
exp_root = completed_list[0]
print("Working on data from:", exp_root)

res_by_vel_zset_key = "%s:res_by_vel_zset" % (exp_root)
res_by_vol_zset_key = "%s:res_by_vol_zset" % (exp_root)
row_noz_pair_set_key = "%s:row_noz_pair_set" % (exp_root)


Working on data from: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A


In [106]:
# Set the velocity desired range (mps)
#
vel_desired_range = (5, 6.5)
#vol_desired_range = (vol_max_bin-(vol_scores_np.std()/range_div), vol_max_bin+(vol_scores_np.std()/range_div))

In [107]:
# Fetch results which fall withing desired velocity range
#
low  = str(vel_desired_range[0])
high = str(vel_desired_range[1])
print(low, high)
vel_results_desired_range = r.zrangebyscore(res_by_vel_zset_key, low, high, withscores=True)
len(vel_results_desired_range)
vel_results_desired_range[0]

5 6.5


('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:253:4192', 5.000465)

In [50]:

row_noz_pair_list = list(r.smembers(row_noz_pair_set_key))

vol_bin_data_dict = {}
vel_bin_data_dict = {}

for row_noz_pair in row_noz_pair_list:
    print("Fetching and separating data for row_nozzle pair:", row_noz_pair)
    vol_bin_data_dict[row_noz_pair] = separate_results_into_bins(r, res_by_vol_zset_key, num_bins, row_noz_pair)
    vel_bin_data_dict[row_noz_pair] = separate_results_into_bins(r, res_by_vel_zset_key, num_bins, row_noz_pair)

Working on data from  2017-10-02_08.42.46:1:SNX1227354:14:FF031A
Fetching and separating data for row_nozzle pair: 1:247
Fetching and separating data for row_nozzle pair: 1:250
Fetching and separating data for row_nozzle pair: 1:252
Fetching and separating data for row_nozzle pair: 1:251
Fetching and separating data for row_nozzle pair: 1:248
Fetching and separating data for row_nozzle pair: 1:249
Fetching and separating data for row_nozzle pair: 1:253


In [98]:
print("Results scored by volume")
for row_noz_pair, bd in vol_bin_data_dict.items():
    print("row_nozzle pair:", row_noz_pair)
    for idx, ((low, high), hist, count) in enumerate(zip(bd.bin_ranges, bd.hist_bins, bd.bin_set_counts)):
        print(f"  [bin {idx+1:02}] range: {low:.3}-{high:.3}, count: {count}, hist: {hist:.3}")

print("Results scored by velocity")
for row_noz_pair, bd in vel_bin_data_dict.items():
    print("row_nozzle pair:", row_noz_pair)
    for idx, ((low, high), hist, count) in enumerate(zip(bd.bin_ranges, bd.hist_bins, bd.bin_set_counts)):
        print(f"  [bin {idx+1:02}] range: {low:.3}-{high:.3}, count: {count}, hist: {hist:.3}")


Results scored by volume
row_nozzle pair: 1:247
  [bin 01] range: 0.249-0.284, count: 103, hist: 0.266
  [bin 02] range: 0.284-0.318, count: 84, hist: 0.301
  [bin 03] range: 0.318-0.353, count: 75, hist: 0.335
  [bin 04] range: 0.353-0.387, count: 65, hist: 0.37
  [bin 05] range: 0.387-0.422, count: 59, hist: 0.405
  [bin 06] range: 0.422-0.456, count: 34, hist: 0.439
  [bin 07] range: 0.456-0.491, count: 35, hist: 0.474
  [bin 08] range: 0.491-0.526, count: 33, hist: 0.508
  [bin 09] range: 0.526-0.56, count: 43, hist: 0.543
  [bin 10] range: 0.56-0.595, count: 61, hist: 0.578
  [bin 11] range: 0.595-0.629, count: 62, hist: 0.612
  [bin 12] range: 0.629-0.664, count: 64, hist: 0.647
  [bin 13] range: 0.664-0.699, count: 68, hist: 0.681
  [bin 14] range: 0.699-0.733, count: 70, hist: 0.716
  [bin 15] range: 0.733-0.768, count: 59, hist: 0.75
  [bin 16] range: 0.768-0.802, count: 55, hist: 0.785
  [bin 17] range: 0.802-0.837, count: 37, hist: 0.82
  [bin 18] range: 0.837-0.872, count: 

In [37]:
print(vol_bin_data.index)

<bound method Sequence.index of BinData(num_bins=30, bin_size=0.4000490666666667, bin_sets=[[('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:253:2050', 0.624399), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2180', 0.64139), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2950', 0.660108), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:251:970', 0.661671), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:253:1935', 0.696867), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2949', 0.697365), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2820', 0.699141), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2179', 0.712571), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:251:2955', 0.723722), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2050', 0.731426), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:248:2950', 0.747082), ('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:249:2818', 0.754097), ('2017-10-02_08.42.46:1:SNX122735

In [22]:
volBinData = BinData(30, 67)
volBinData.bin_sets.append(45)
print(volBinData)

BinData(num_bins=30, bin_size=67, bin_sets=[45], bin_ranges=[], hist_bins=[], bin_set_counts=[], cnt=None)


In [27]:
volBinData = BinData2(30, 67)
volBinData.bin_ranges.append(34)
print(volBinData.bin_ranges)

[34]


### Sandbox

In [90]:
d = vol_bin_data_dict['1:251']
'''     bin_data.hist_bins.append( low+((high-low)/2.0)  )
        bin_data.bin_ranges.append( (low, high) )
        bin_data.bin_sets.append( zrange_lst_filtered )
        bin_data.bin_set_counts.append( len(zrange_lst_filtered) )
'''
for (a,d),b,c in zip(d.bin_ranges, d.bin_sets, d.hist_bins):
    print(a,c,d)


0.248978 0.2662706166666667 0.28356323333333333
0.28356323333333333 0.30085585000000004 0.3181484666666667
0.3181484666666667 0.33544108333333333 0.35273370000000004
0.35273370000000004 0.37002631666666674 0.3873189333333334
0.38731893333333334 0.40461155000000004 0.4219041666666667
0.42190416666666664 0.43919678333333334 0.4564894
0.45648940000000005 0.47378201666666675 0.4910746333333334
0.49107463333333334 0.50836725 0.5256598666666666
0.5256598666666666 0.5429524833333332 0.5602450999999999
0.5602451 0.5775377166666666 0.5948303333333333
0.5948303333333333 0.61212295 0.6294155666666666
0.6294155666666666 0.6467081833333332 0.6640008
0.6640008000000001 0.6812934166666667 0.6985860333333334
0.6985860333333334 0.7158786500000001 0.7331712666666667
0.7331712666666667 0.7504638833333332 0.7677565
0.7677565000000001 0.7850491166666667 0.8023417333333334
0.8023417333333334 0.8196343500000001 0.8369269666666667
0.8369269666666667 0.8542195833333333 0.8715122
0.8715122000000001 0.8888048166

In [23]:
class BinData2():
    def __init__(self, num_bins, bin_size):
        self.
        self.bin_sets = []
        self.bin_ranges = []
        self.hist_bins = []
        self.bin_set_counts = []
        self.cnt = 0
